Here's a lesson on creating Multi-Agent Teams with AutoGen:

# Creating Multi-Agent Teams with AutoGen

## Setup

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Introduction

Multi-agent teams in AutoGen are groups of AI agents that collaborate to accomplish complex tasks. Teams provide structure for agent interactions and help coordinate their efforts effectively.

## Basic Team Creation

Let's create a simple team with two agents - a primary agent and a critic:

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4-turbo-preview"
)

# Create agents
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a creative writer who writes engaging content."
)

critic_agent = AssistantAgent(
    "critic", 
    model_client=model_client,
    system_message="You provide constructive feedback on writing. Respond with 'APPROVE' when satisfied."
)

# Create termination condition
termination = TextMentionTermination("APPROVE")

# Create team
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=termination
)

## Running the Team

There are two main ways to run a team:

### 1. Basic Run

In [ ]:
# Run team synchronously
result = await team.run(task="Write a short story about a magical forest.")

### 2. Streaming Run

In [ ]:
# Stream messages as they're generated
async for message in team.run_stream(task="Write a short story about a magical forest."):
    if isinstance(message, TaskResult):
        print(f"Task completed: {message.stop_reason}")
    else:
        print(f"{message.source}: {message.content}")

## Team Types

AutoGen provides several team types for different use cases:

### 1. RoundRobinGroupChat
Agents take turns in a fixed order. Good for structured discussions.

In [ ]:
team = RoundRobinGroupChat(
    [agent1, agent2, agent3],
    termination_condition=termination
)

### 2. SelectorGroupChat 
Uses an AI model to dynamically choose the next speaker based on context.

In [ ]:
team = SelectorGroupChat(
    participants=[agent1, agent2, agent3],
    model_client=model_client,
    termination_condition=termination
)

### 3. Swarm
Agents communicate through explicit handoffs, allowing targeted interactions.

In [ ]:
team = Swarm(
    [agent1, agent2],
    termination_condition=termination
)

## Team Control

### Stopping a Team
Use ExternalTermination to stop a team from outside:

In [ ]:
from autogen_agentchat.conditions import ExternalTermination

external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=external_termination
)

# Run team in background
task = asyncio.create_task(team.run(task="Some task"))

# Stop team after some condition
await asyncio.sleep(5)
external_termination.set()

### Resetting a Team
Reset team state between different tasks:

In [ ]:
await team.reset()

### Resuming a Team
Continue previous conversation:

In [ ]:
# First run
await team.run(task="Start a story about space exploration")

# Continue the conversation
await team.run(task="Add a plot twist to the story")

## Best Practices

1. Choose the right team type for your use case:
   - RoundRobinGroupChat for structured, turn-based interactions
   - SelectorGroupChat for dynamic, context-aware speaker selection
   - Swarm for explicit handoffs between agents

2. Use appropriate termination conditions:
   - TextMentionTermination for keyword-based stopping
   - MaxMessageTermination for limiting conversation length
   - ExternalTermination for manual control

3. Reset teams between unrelated tasks to clear conversation history

4. Use streaming for real-time monitoring of team interactions

5. Start with simple teams and add complexity as needed

Remember that teams add overhead compared to single agents, so use them only when the task complexity justifies multi-agent collaboration.